Домашнее задание №3 Fine-tune model from huggingface.co. Придумать интересную задача, найти претреейн модель, fine-tune/allign. Показать демо приложения (20 баллов), сложные варианты fine-tune типа RLHF +20 баллов.

Выполнила: Винницкая Диана

Формулировка задачи: генерация описания фильма по названию и жанру на русском языке

Датасет с сайта kaggle:

https://www.kaggle.com/datasets/maksimpotorochin/movie-plots-from-wikipedia-in-russian

Необходимые этапы:
1. подготовка данных
2. выбор модели
4. токенизация
5. fine-tuning

Загрузка датасета:

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("maksimpotorochin/movie-plots-from-wikipedia-in-russian")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/movie-plots-from-wikipedia-in-russian


In [4]:
import pandas as pd
df=pd.read_csv('/kaggle/input/movie-plots-from-wikipedia-in-russian/films_data.csv', encoding='utf-8')
df.head()

,title,type,genre,imdb_rating,summary,plot
0,? (фильм),film,драматический фильм,7.0,«?» (индон. Tanda Tanya) — индонезийский худож...,Основная тема фильма — межрелигиозные отношени...
1,...а пятый всадник – Страх,film,драма военный артхаус,7.2,«…а пятый всадник — Страх» (чеш. …a páty jezde...,Прага во время немецкой оккупации Чехословакии...
2,…и передайте привет ласточкам,film,драма военный,6.8,«…и передайте привет ласточкам» — (чеш. ...a p...,"Конец 1942 года, нацистская тюрьма в Бреслау. ..."
3,«Чудотворец» из Бирюлёва,film,игровое кино,NaN,«Чудотворец из Бирюлёва» — советский короткоме...,Жена ответственного работника Зоя Фёдоровна на...
4,(Не)идеальные роботы,film,комедия фантастика,5.4,«(Не)идеальные роботы» (англ. Robots) — художе...,Действие фильма разворачивается в далёком буду...


Предобработка: удаляем строки, где пустое описание или жанр, создаем пары

In [12]:
from datasets import Dataset
df = df[df['plot'].notnull() & df['genre'].notnull() & df['title'].notnull()]

# Создаём пары (imput (название, жанр) и target (описание))
df['input_text'] = df.apply(lambda row: f"Название: {row['title']}; Жанр: {row['genre']}", axis=1)
df['target_text'] = df['plot']

dataset = Dataset.from_pandas(df[['input_text', 'target_text']])

In [13]:
dataset = dataset.shuffle(seed=42).select(range(5000))

Я выбрала модель предобученную на русском языке: ai-forever/ruT5-base

Нужно токенизировать текст

In [15]:
from transformers import AutoTokenizer

model_name = "ai-forever/ruT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(example):
    model_inputs = tokenizer(
        example["input_text"],
        max_length=256,
        padding="max_length",
        truncation=True,
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["target_text"],
            max_length=256,
            padding="max_length",
            truncation=True,
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [16]:
import torch
print(torch.cuda.is_available())

True


In [17]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer

# загрузка модели
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# аргументы
training_args = TrainingArguments(
    output_dir="./movie_model",
    per_device_train_batch_size=16,
    num_train_epochs=5,
    learning_rate=5e-5,
    logging_steps=100,
    save_total_limit=1,
    save_strategy="epoch",
    report_to="none",
    fp16=True if torch.cuda.is_available() else False,
)

# Тренер
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [18]:
trainer.train()

Step,Training Loss
100,4.705600
200,2.965100
300,2.854900
400,2.807100
500,2.755600
600,2.727700
700,2.671800
800,2.672200
900,2.651600
1000,2.674600


TrainOutput(global_step=1565, training_loss=2.8238856696473142, metrics={'train_runtime': 1393.2362, 'train_samples_per_second': 17.944, 'train_steps_per_second': 1.123, 'total_flos': 7611973632000000.0, 'train_loss': 2.8238856696473142, 'epoch': 5.0})

In [23]:
trainer.save_model("./movie_model")

Пример описания:

In [19]:
def generate_description(title, genre, max_tokens=100):
    prompt = f"Название: {title}; Жанр: {genre}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.3,
        num_return_sequences=1,
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [20]:
desc = generate_description("Солярис", "фантастика, драма")
print("Описание:", desc)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Описание: Действие фильма разворачивается в 1991 году. В рамках проекта проведён проект «Солярис» (Вектор Лесли) и его главный герой — профессиональный разведчик, который не может работать на земле из-за своей внешности.


In [21]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.8 MB/s eta 0:00:00


In [24]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "ai-forever/ruT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained("./movie_model")  # путь к дообученной модели

def generate_description(title, genre):
    prompt = f"Название: {title}; Жанр: {genre}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.2
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

gr.Interface(
    fn=generate_description,
    inputs=[
        gr.Textbox(label="Название фильма"),
        gr.Textbox(label="Жанр"),
    ],
    outputs=gr.Textbox(label="Сгенерированное описание"),
    title="Генератор описания фильма"
).launch(share=True)  # ссылка

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9f58528e28732554eb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
